In [339]:
import numpy as np

class Quaternion():
    def __init__(self, w, x, y, z):
        self.quat = np.array([w, x, y, z])

    @classmethod
    def from_array(cls, quat):
        if len(quat) == 3:
            return cls(0, quat[0], quat[1], quat[2])
        else:
            return cls(quat[0], quat[1], quat[2], quat[3])

    @classmethod
    def from_rotation(cls, vec, theta):
        vec = np.array(vec).reshape(3)
        vec = vec/np.linalg.norm(vec)
        t = theta / 2
        s = np.sin(t)
        c = np.cos(t)
        return cls(c, vec[0]*s, vec[1]*s, vec[2]*s)    

    def __array__(self) -> np.ndarray:
        return self.quat
        
    def as_array(self):
        return self.quat

    def as_vector(self):
        return self.as_array().reshape(4, 1)

    def as_matrix(self):
        q = self.as_array()
        return np.array([
                    [q[0], -q[1], -q[2], -q[3]],
                    [q[1],  q[0], -q[3],  q[2]],
                    [q[2],  q[3],  q[0], -q[1]],
                    [q[3], -q[2],  q[1],  q[0]],
                ])

    def as_rotation_matrix(self):
        w, x, y, z = self.as_array()
        return np.array([
                    [w*w + x*x - y*y - z*z, 2*(x*y - w*z), 2*(w*y + x*z)], 
                    [2*(w*z + x*y), w*w - x*x + y*y - z*z, 2*(-w*x + y*z)], 
                    [2*(x*z - w*y), 2*(y*z + w*x), w*w - x*x - y*y + z*z], 
                ])

    def conjugate(self):
        w, x, y, z = self.as_array()
        return Quaternion(w, -x, -y, -z)

    def norm(self, ord=2):
        return np.linalg.norm(self.as_array(), ord=ord)

    def normalize(self):
        return self * (1.0 / self.norm())

    def inverse(self):
        return self.conjugate() * (1.0 / self.norm()**2)
    
    def __str__(self):
        return str(self.quat)

    def __repr__(self):
        return str(self.quat)

    def __pos__(self):
        return Quaternion.from_array(self.as_array())

    def __neg__(self):
        return Quaternion.from_array(-self.as_array())

    def __add__(self, other):
        if type(other) == Quaternion: 
            return Quaternion.from_array(self.as_array() + other.as_array())
        else:
            return Quaternion.from_array(self.as_array() + other)
    
    def __sub__(self, other):
        if type(other) == Quaternion: 
            return Quaternion.from_array(self.as_array() - other.as_array())
        else:
            return Quaternion.from_array(self.as_array() - other)
        
    def __mul__(self, other):
        if type(other) == Quaternion: 
            w0, x0, y0, z0 = self.as_array()
            w1, x1, y1, z1 = other.as_array()
            return Quaternion(
                    w0*w1 - x0*x1 - y0*y1 - z0*z1,
                    x0*w1 + w0*x1 - z0*y1 + y0*z1,
                    y0*w1 + z0*x1 + w0*y1 - x0*z1,
                    z0*w1 - y0*x1 + x0*y1 + w0*z1,
                )
        else:
            return Quaternion.from_array(self.as_array() * other)


In [341]:
# scipy と比較してみる
from scipy.spatial.transform import Rotation
euler = np.array([np.deg2rad(10),  np.deg2rad(20), np.deg2rad(30)])
rot = Rotation.from_euler('XYZ', euler)
print('scipy')
print(rot.as_matrix())

rot0 = Rotation.from_euler('XYZ', np.array([np.deg2rad(10), np.deg2rad(0), np.deg2rad(0)]))
rot1 = Rotation.from_euler('XYZ', np.array([np.deg2rad(0), np.deg2rad(20), np.deg2rad(0)]))
rot2 = Rotation.from_euler('XYZ', np.array([np.deg2rad(0), np.deg2rad(0), np.deg2rad(30)]))
#print(rot0.as_quat())
#print(rot1.as_quat())
#print(rot2.as_quat())
print((rot0*rot1*rot2).as_matrix())

q0 = Quaternion.from_rotation([1, 0, 0], np.deg2rad(10))
q1 = Quaternion.from_rotation([0, 1, 0], np.deg2rad(20))
q2 = Quaternion.from_rotation([0, 0, 1], np.deg2rad(30))
q = q0*q1*q2
print('my class')
print(q.as_rotation_matrix())


scipy
[[ 0.81379768 -0.46984631  0.34202014]
 [ 0.54383814  0.82317294 -0.16317591]
 [-0.20487413  0.31879578  0.92541658]]
[[ 0.81379768 -0.46984631  0.34202014]
 [ 0.54383814  0.82317294 -0.16317591]
 [-0.20487413  0.31879578  0.92541658]]
my class
[[ 0.81379768 -0.46984631  0.34202014]
 [ 0.54383814  0.82317294 -0.16317591]
 [-0.20487413  0.31879578  0.92541658]]


In [342]:
from socket import socket, AF_INET, SOCK_DGRAM
def send_udp(q):
    S_PORT = 4002
    ADDRESS = "172.17.32.1"
    ss = socket(AF_INET, SOCK_DGRAM)
    m = q.as_rotation_matrix()
    s_msg = str(m[0][0])+','+str(m[0][1])+','+str(m[0][2])+','+str(m[1][0])+','+str(m[1][1])+','+str(m[1][2])+','+str(m[2][0])+','+str(m[2][1])+','+str(m[2][2])
    ss.sendto(s_msg.encode(), (ADDRESS, S_PORT))
    ss.close()

In [343]:

p = Quaternion(1, 0, 0, 0)

dt = 0.1
q0 = Quaternion.from_rotation([1, 0, 0], dt*np.deg2rad(20))
q1 = Quaternion.from_rotation([0, 1, 0], dt*np.deg2rad(10))
q2 = Quaternion.from_rotation([0, 0, 1], dt*np.deg2rad(0))
q = q0*q1*q2
import time
for i in range(200):
    send_udp(p)
    
    p = (q*dt)*p
    p = p.normalize()
    time.sleep(0.01)

In [344]:
# 適当な方向に向けてみるテスト
send_udp(Quaternion.from_rotation([0, 0, 1], np.deg2rad(45)))

In [345]:
# 基準から回す
p = Quaternion(1, 0, 0, 0)
q = Quaternion.from_rotation([0, 1, 1], np.deg2rad(45))
p = q * p
send_udp(p)

In [346]:
# 現在の姿勢を基準に回す
q = Quaternion.from_rotation([0, 1, 0], np.deg2rad(45))
q1 = p * q * p.conjugate() # 座標系変換
p1 = q1 * p
send_udp(p1)

In [347]:
# 現在の姿勢を基準に回す
q = Quaternion.from_rotation([0, 0, 1], np.deg2rad(45))
q2 = p1 * q * p1.conjugate() # 座標系変換
p2 = q2 * p1
send_udp(p2)

In [348]:
#p = q.inverse() * p
#send_udp(p)

In [349]:
v = Quaternion(0, 1, 0, 0)
q = Quaternion.from_rotation([0, 0, 1], np.deg2rad(90))
q * v * q.conjugate()

[0.00000000e+00 2.22044605e-16 1.00000000e+00 0.00000000e+00]